In [ ]:
from scapy.all import *
from hashlib import md5
import arrow
from parse_rssi import parse_rssi

In [ ]:
import glob
capture_files = glob.glob("captured_packets/*.cap")

## Get packets from all available collects
### Probe requests are management frames of type 0x00 with subtype 0x04

In [ ]:
collects = []
for capture_file in capture_files:
    pcap_reader = PcapReader(capture_file)
    packets = []
    for pkt in pcap_reader:
        try:
            if (pkt.type == 0 and pkt.subtype == 0x04):
                packets.append(pkt)
        except:
            pass
    collects.append((capture_file, packets))

# Print out the start and end times of the collected packages

In [ ]:
for collect in collects:
    packets = collect[1]
    pkt_start = packets[1]
    pkt_end = packets[-1]
    
    print(f'Station: {collect[0]}')
    print(f'Start: {arrow.get(pkt_start.time)}')
    print(f'End  : {arrow.get(pkt_end.time)}\n')

## Find the MAC address from each packet, it's associated RSSI (signal strength) and the time of capture

In [ ]:
for num, collect in enumerate(collects):
    packets = collect[1]
    near = [((pkt.addr2), arrow.get(pkt.time), parse_rssi(pkt)) for pkt in packets]
    collects[num] = collect + (near,)

## Save the data on a CSV

In [ ]:
import csv
with open('parsed_packet_data.csv','w') as f:
    wr = csv.writer(f, dialect='excel')
    wr.writerows([('MAC', 'time', 'RSSI', 'station')])
    
    # Write on collect at a time
    for collect in collects:
        # Iterate through the proximity data of the current collect
        for near in collect[2]:
            # Add the station (ID) data to each proximity reading
            near = near + (collect[0],)
            wr.writerow(near)